# L1 regression of mean energy consumption

In [52]:
 # data and numbers
import numpy as np
import pandas as pd
import datetime as dt

# Visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import ticker
import matplotlib.dates as mdates
%matplotlib inline

import pandas as pd
from sklearn import preprocessing

In [53]:
path_raw = "..//data//meters//raw//"
path_meta = "..//data//metadata//"
path_cleaned = "..//data//meters//cleaned//"

In [54]:
columns_to_use = [
    "building_id",
    "site_id",
    "sqm",
    "lat",
    "lng",
    "timezone",
    "industry",
    "subindustry",
    "heatingtype",
    "yearbuilt",
    "date_opened",
    "numberoffloors",
]

In [55]:
meta = pd.read_csv(
    path_meta + "metadata.csv",
    usecols=columns_to_use,
    index_col="building_id"
)
meta.info()
meta.head()

def import_data(metername):
    # load data
    df = pd.read_csv(path_cleaned + metername + ".csv")
    
    # Transform timestamp to datetime object type
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S")
    df.set_index("timestamp", inplace=True)
    df.info()
    return df

sensor_data = import_data("electricity_cleaned")

# from meta, drop rows that are not in sensor_data index
meta = meta[meta.index.isin(sensor_data.columns)]

# split meta into train and test with sklearn
from sklearn.model_selection import train_test_split
train, test = train_test_split(meta, test_size=0.2, random_state=42)

<class 'pandas.core.frame.DataFrame'>
Index: 1636 entries, Panther_lodging_Dean to Mouse_science_Micheal
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   site_id         1636 non-null   object 
 1   sqm             1636 non-null   float64
 2   lat             1399 non-null   float64
 3   lng             1399 non-null   float64
 4   timezone        1636 non-null   object 
 5   industry        579 non-null    object 
 6   subindustry     579 non-null    object 
 7   heatingtype     215 non-null    object 
 8   yearbuilt       817 non-null    float64
 9   date_opened     21 non-null     object 
 10  numberoffloors  441 non-null    float64
dtypes: float64(5), object(6)
memory usage: 153.4+ KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17544 entries, 2016-01-01 00:00:00 to 2017-12-31 23:00:00
Columns: 1578 entries, Panther_parking_Lorriane to Mouse_science_Micheal
dtypes: float64(1578)
memory usage: 211

In [56]:
mean_sensor_data = sensor_data.mean()
mean_sensor_data.info()
print(mean_sensor_data)

<class 'pandas.core.series.Series'>
Index: 1578 entries, Panther_parking_Lorriane to Mouse_science_Micheal
Series name: None
Non-Null Count  Dtype  
--------------  -----  
1572 non-null   float64
dtypes: float64(1)
memory usage: 89.2+ KB
Panther_parking_Lorriane      10.722723
Panther_lodging_Cora         135.054565
Panther_office_Hannah          6.683938
Panther_lodging_Hattie       140.593488
Panther_education_Teofila    156.466222
                                ...    
Mouse_lodging_Vicente         45.767561
Mouse_health_Justin          783.350343
Mouse_health_Ileana           43.587324
Mouse_health_Estela          384.684218
Mouse_science_Micheal        435.844964
Length: 1578, dtype: float64


In [57]:
# add mean_sensor_data as column to train_meta by index
print("Train")
train['mean_sensor_data'] = mean_sensor_data
train.info()

print("Test")
test['mean_sensor_data'] = mean_sensor_data
test.info()

Train
<class 'pandas.core.frame.DataFrame'>
Index: 1262 entries, Fox_office_Brandy to Bobcat_lodging_Melaine
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   site_id           1262 non-null   object 
 1   sqm               1262 non-null   float64
 2   lat               1070 non-null   float64
 3   lng               1070 non-null   float64
 4   timezone          1262 non-null   object 
 5   industry          463 non-null    object 
 6   subindustry       463 non-null    object 
 7   heatingtype       177 non-null    object 
 8   yearbuilt         617 non-null    float64
 9   date_opened       15 non-null     object 
 10  numberoffloors    353 non-null    float64
 11  mean_sensor_data  1257 non-null   float64
dtypes: float64(6), object(6)
memory usage: 128.2+ KB
Test
<class 'pandas.core.frame.DataFrame'>
Index: 316 entries, Hog_office_Alisha to Mouse_health_Modesto
Data columns (total 12 columns):
 #   Col

In [60]:
# convert categorical columns to one-hot encoding
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# drop NaN values
train.dropna(inplace=True)
test.dropna(inplace=True)

# Train linear regression model using L1 regularization on dataframe train, where column mean_sensor_data is the target
from sklearn.linear_model import Lasso
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(train.drop(columns=["mean_sensor_data"]), train["mean_sensor_data"])

/home/stefkoma/anaconda3/envs/llm_tutorial/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.323e+05, tolerance: 1.364e+03
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)